In [4]:
import os

# pensimpy

In [5]:
from pensimpy.examples.recipe import Recipe, RecipeCombo
from pensimpy.data.constants import FS, FOIL, FG, PRES, DISCHARGE, WATER, PAA
from pensimpy.data.constants import FS_DEFAULT_PROFILE, FOIL_DEFAULT_PROFILE, FG_DEFAULT_PROFILE, \
    PRESS_DEFAULT_PROFILE, DISCHARGE_DEFAULT_PROFILE, WATER_DEFAULT_PROFILE, PAA_DEFAULT_PROFILE
from quarticgym.envs.pensimenv import PenSimEnvGym, PeniControlData, NUM_STEPS

In [10]:
# set up the default recipe
recipe_dict = {FS: Recipe(FS_DEFAULT_PROFILE, FS),
            FOIL: Recipe(FOIL_DEFAULT_PROFILE, FOIL),
            FG: Recipe(FG_DEFAULT_PROFILE, FG),
            PRES: Recipe(PRESS_DEFAULT_PROFILE, PRES),
            DISCHARGE: Recipe(DISCHARGE_DEFAULT_PROFILE, DISCHARGE),
            WATER: Recipe(WATER_DEFAULT_PROFILE, WATER),
            PAA: Recipe(PAA_DEFAULT_PROFILE, PAA)}

recipe_combo = RecipeCombo(recipe_dict=recipe_dict)
# set up the environment
env = PenSimEnvGym(recipe_combo=recipe_combo)
state = env.reset()
# load one batch of the sampled data
load_just_a_file='quarticgym/datasets/pensimenv/random_batch_0.csv'
dataset_obj = PeniControlData(load_just_a_file=load_just_a_file)
if dataset_obj.file_list:
    print('Penicillin_Control_Challenge data correctly initialized.')
else:
    raise ValueError("Penicillin_Control_Challenge data initialization failed.")
dataset = dataset_obj.get_dataset()
# need to be set by dataset for normalization
env.max_observations = dataset_obj.max_observations
env.min_observations = dataset_obj.min_observations
env.normalize = True
# now, let's try to run for one epoch, use the actions cloned from a csv file. Note that the environment setup might be different.
total_reward = 0.0
for step in range(NUM_STEPS):
    state, reward, done, info = env.step(dataset['actions'][step].tolist())
    total_reward += reward
    if step%1000 == 0:
        print("reward, total_reward:",reward, total_reward)
print("your total reward is (by default, should be 2965.6227):", total_reward)

Penicillin_Control_Challenge data correctly initialized.
reward, total_reward: 0.05834041883349912 0.05834041883349912
reward, total_reward: 2.7455784447631686 2586.295427653821
your total reward is (by default, should be 2965.6227): 2965.6087902382847
